In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the last data point in the database
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

In [ ]:
# Perform a query to retrieve the date and precipitation data
# Include only most recent year
# Sort results by date ascending(default)
results = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > '2016-08-23').\
    order_by(Measurement.date).all()

#Load into pandas dataframe, use date as index
precip_df = pd.DataFrame(results, columns=['date', 'prcp'])
precip_df.set_index('date', inplace=True)

#replace NANs with 0 in prcp column
precip_df = precip_df.fillna(0)

precip_df.head()



In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
precip_df.plot(y='prcp', title="Precipitation in Hawaii", figsize=(8,6))
plt.tick_params(labelbottom=False) 

plt.show()

In [ ]:
#code for inspecting data while writing the code. No need to run this cell for grading
first_row = session.query(Measurement).first()
first_row.__dict__

inspector = inspect(engine)
inspector.get_table_names()

columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

![precipitation](Images/precipitation.png)

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data

precip_df.describe()

![describe](Images/describe.png)

In [ ]:
# Design a query to show how many stations are available in this dataset?
no_of_stations = engine.execute("SELECT count(id) as station_count FROM station")
for row in no_of_stations: print(f'There are {row[0]} stations.')

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
most_active = engine.execute("SELECT s.station, count(m.id)\
                                as m_count FROM measurement m\
                                INNER JOIN station s on m.station = s.station\
                                GROUP BY s.station ORDER BY m_count DESC")
for row in most_active: print(row)

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
most_active_temps = pd.read_sql_query("SELECT min(m.tobs) as min_temp, max(m.tobs) as max_temp, round(avg(m.tobs),1) as avg_temp \
                                 FROM measurement m \
                                 JOIN station s \
                                 ON m.station = s.station \
                                 WHERE s.station = 'USC00519281'", engine)
most_active_temps

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
t_most_temp_data = pd.read_sql_query("SELECT m.tobs, m.date, s.station \
                                 FROM measurement m \
                                 JOIN station s \
                                 ON m.station = s.station \
                                 WHERE m.date BETWEEN '2016-08-23' and '2017-08-23' \
                                 AND s.station = 'USC00519281'", engine)
t_most_temp_data.plot(y='tobs', kind='hist', bins=12)
plt.title("Observed temperatures - Station USC00519281")
plt.xlabel("Temperature (F)")

![precipitation](Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
prev_year_stats = calc_temps('2017-02-28', '2017-03-05')
print(prev_year_stats)

In [ ]:
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
ptp = prev_year_stats[0][2] - prev_year_stats[0][0]

# Plot the results from your previous query as a bar chart. 
plt.figure(figsize=(2,6))
height = [prev_year_stats[0][1]]
y_pos = np.arange(len(height))
plt.bar(y_pos, height, yerr=ptp/2)
plt.xticks([])

# Use "Trip Avg Temp" as your Title
plt.title("Trip Avg Temp")

# Use the average temperature for the y value
plt.ylabel("Avg Temp (F)")

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

trip_rain = pd.read_sql_query("SELECT sum(m.prcp) as total_rain, s.station, s.name, s.latitude, s.longitude, s.elevation \
                                 FROM measurement m \
                                 JOIN station s \
                                 ON m.station = s.station \
                                 WHERE m.date between '2017-02-28' and '2017-03-05' \
                                 GROUP BY s.station \
                                 ORDER BY total_rain DESC", engine)

trip_rain

## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
